<a href="https://colab.research.google.com/github/danielAdama/tutorials/blob/main/Causual_Lang_Model_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers
!pip install datasets
!pip install boto3
!pip install colab-env --upgrade
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -c "from accelerate.utils import write_basic_config; write_basic_config(mixed_precision='fp16')"

2023-05-08 12:47:34.010601: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


In [ ]:
!accelerate env

2023-05-08 12:47:42.883018: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.18.0
- Platform: Linux-5.10.147+-x86_64-with-glibc2.31
- Python version: 3.10.11
- Numpy version: 1.22.4
- PyTorch version (GPU?): 2.0.0+cu118 (True)
- `Accelerate` default config:
	- compute_environment: LOCAL_MACHINE
	- distributed_type: NO
	- mixed_precision: fp16
	- use_cpu: False
	- num_processes: 1
	- machine_rank: 0
	- num_machines: 1
	- rdzv_backend: static
	- same_network: False
	- main_training_function: main
	- downcast_bf16: False
	- tpu_use_cluster: False
	- tpu_use_sudo: False


In [ ]:
import colab_env
colab_env.__version__
# !more gdrive/My\ Drive/vars.env

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'0.2.0'

In [ ]:
colab_env.envvar_handler.add_env("AWS_SECRET_ACCESS_KEY", "", overwrite=False)
colab_env.envvar_handler.add_env("AWS_ACCESS_KEY_ID", "", overwrite=False)
colab_env.envvar_handler.add_env("AWS_BUCKET_NAME", "", overwrite=False)
colab_env.envvar_handler.add_env("AWS_MODEL_BUCKET_NAME", "", overwrite=False)

# !more gdrive/My\ Drive/vars.env

In [ ]:
import os
from collections import defaultdict
from tqdm import tqdm
from datasets import (Dataset, load_dataset, DatasetDict)
import boto3
from transformers import (
    AutoTokenizer, 
    GPT2Tokenizer, 
    GPT2LMHeadModel, 
    AutoConfig, 
    DataCollatorForLanguageModeling, 
    Trainer, 
    TrainingArguments
)
import json
import math

In [ ]:
 s3 = boto3.client(
    "s3",
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
    )

In [ ]:
def load_jsonl():
  
  ds_valid = []
  ds_train = []

  count = 0
  remoteObjs = s3.list_objects_v2(Bucket=os.getenv("AWS_BUCKET_NAME"))
  for obj in remoteObjs['Contents']:
    count += 1
    path = obj['Key']
    if count == 1:
      trainobj = s3.get_object(Bucket=os.getenv("AWS_BUCKET_NAME"), Key=path)
      body = trainobj['Body'].read().decode('utf-8')
      lines = body.split('\n')
      for line in lines:
          if line.strip() != '':
              ds_train.append(json.loads(line))
    else:
      validobj = s3.get_object(Bucket=os.getenv("AWS_BUCKET_NAME"), Key=path)
      body = validobj['Body'].read().decode('utf-8')
      lines = body.split('\n')
      for line in lines:
          if line.strip() != '':
              ds_valid.append(json.loads(line))

  return ds_train, ds_valid

In [ ]:
ds_train, ds_valid = load_jsonl()

**The script below is to load data in chucks locally!**

In [ ]:
# import os
# import pandas as pd
# from tqdm import tqdm
# import json


# train = 'codeparrot-ds-train.jsonl'
# sub = 'codeparrot-ds-sub-train.jsonl'
# data = r'/home/chuby/Desktop/programming/NLP/gpt-2-train/datasets'

# # with open(os.path.join(data,'codeparrot-ds-valid.jsonl'), 'r') as docs:
#     # for line in docs:
#     #     test_ds.append(json.loads(line))

# # print(test_dict)

# num_lines = sum(1 for line in open(os.path.join(data,train)))
# print(f'Num. of samples in train dataset: {num_lines:,}')

# # with open(data) as infile:
# #   o = json.load(infile)
# #   chunkSize = 1000
# #   for i in xrange(0, len(o), chunkSize):
# #     with open('file_' + str(i//chunkSize) + '.jsonl', 'wb') as outfile:
# #       json.dump(o[i:i+chunkSize], outfile)


# chunksize = 1000
# chunks = pd.read_json(os.path.join(data,train), lines=True, chunksize=chunksize)
# sub_data = []
# for chunk in tqdm(chunks):
#     sub_data.append(chunk)
#     if len(sub_data) == 3:
#        break

# test_ds = []
# interactions = pd.concat(sub_data, ignore_index=True, sort=True)
# interactions = interactions.reindex(columns=['repo_name','path', 'copies', 'size', 'content', 'license'])
# print(interactions.head())
# print(interactions.shape)
# interactions.to_json(os.path.join(data,sub), orient='records', lines=True)
# with open(os.path.join(data,sub), 'r') as docs:
#     for line in docs:
#         test_ds.append(json.loads(line))

# print(test_ds)

In [ ]:
def any_keyword_in_string(string: str, keywords):
  return any(keyword in string for keyword in keywords)

def filter_streaming_dataset(dataset, filters):
  filter_dict = defaultdict(list)
  total = 0
  for sample in tqdm(iter(dataset)):
    total += 1
    if any_keyword_in_string(sample["content"], filters):
      for k, v in sample.items():
        filter_dict[k].append(v)
        print(f"{len(filter_dict['content'])/total:.2%} of data after filtering.")
  return Dataset.from_dict(filter_dict)

In [ ]:
os.getenv("AWS_SECRET_ACCESS_KEY")filters = ["pandas", "sklearn", "matplotlib", "seaborn"]
split = "train"  # "valid"

example_1 = "import numpy as np"
example_2 = "import pandas as pd"

print(any_keyword_in_string("k", "keywords"))

print(
    any_keyword_in_string(example_1, filters), any_keyword_in_string(example_2, filters)
)

True
False True


The cell below will take a long time to execute, so we will skip it

In [ ]:
# data = load_dataset(f"transformersbook/codeparrot-{split}", split=split, streaming=True)
# filtered_data = filter_streaming_dataset(data, filters)

In [ ]:
# ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train")
# ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")

In [ ]:
# ds_valid = []
# ds_train = []

# with open(os.path.join('huggingface-course','codeparrot-ds-valid.jsonl'), 'r') as docs:
#       for line in docs:
#         ds_valid.append(json.loads(line))

# with open(os.path.join('huggingface-course','codeparrot-ds-sub-train.jsonl'), 'r') as docs:
#       for line in docs:
#         ds_train.append(json.loads(line))

In [ ]:
ds_valid[0]

{'repo_name': 'ratnania/pigasus',
 'path': 'doc/manual/include/demo/test_neumann_quartcircle.py',
 'copies': '1',
 'size': '2730',
 'content': '#! /usr/bin/python\n\n# ...\ntry:\n    from matplotlib import pyplot as plt\n    PLOT=True\nexcept ImportError:\n    PLOT=False\n# ...\nimport numpy                as np\nfrom pigasus.gallery.poisson import *\nimport sys\nimport inspect\nfilename = inspect.getfile(inspect.currentframe()) # script filename (usually with path)\n\n# ...\nsin = np.sin ; cos = np.cos ; pi = np.pi ; exp = np.exp\n# ...\n\n#-----------------------------------\ntry:\n    nx = int(sys.argv[1])\nexcept:\n    nx = 31\n\ntry:\n    ny = int(sys.argv[2])\nexcept:\n    ny = 31\n\ntry:\n    px = int(sys.argv[3])\nexcept:\n    px = 2\n\ntry:\n    py = int(sys.argv[4])\nexcept:\n    py = 2\n\nfrom igakit.cad_geometry import quart_circle as domain\ngeo = domain(n=[nx,ny],p=[px,py])\n#-----------------------------------\n\n# ...\n# exact solution\n# ...\nR = 1.\nr = 0.5\nc = 1. # 

In [ ]:
ds_train[0]

{'repo_name': 'kmike/scikit-learn',
 'path': 'sklearn/utils/__init__.py',
 'copies': 3,
 'size': 10094,
 'content': '"""\nThe :mod:`sklearn.utils` module includes various utilites.\n"""\n\nfrom collections import Sequence\n\nimport numpy as np\nfrom scipy.sparse import issparse\nimport warnings\n\nfrom .murmurhash import murmurhash3_32\nfrom .validation import (as_float_array, check_arrays, safe_asarray,\n                         assert_all_finite, array2d, atleast2d_or_csc,\n                         atleast2d_or_csr, warn_if_not_float,\n                         check_random_state)\nfrom .class_weight import compute_class_weight\n\n__all__ = ["murmurhash3_32", "as_float_array", "check_arrays", "safe_asarray",\n           "assert_all_finite", "array2d", "atleast2d_or_csc",\n           "atleast2d_or_csr", "warn_if_not_float", "check_random_state",\n           "compute_class_weight"]\n\n# Make sure that DeprecationWarning get printed\nwarnings.simplefilter("always", DeprecationWarning)\n\

In [ ]:
valid_dict = {k: [d[k] for d in ds_valid] for k in ds_valid[0]}
train_dict = {k: [d[k] for d in ds_train] for k in ds_train[0]}
valid_set = Dataset.from_dict(valid_dict)
train_set = Dataset.from_dict(train_dict)

print(train_set)
print(valid_set)

Dataset({
    features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
    num_rows: 20000
})
Dataset({
    features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
    num_rows: 3322
})


In [ ]:
raw_dataset = DatasetDict(
    {
        "train": train_set.shuffle(), #.select(range(1000)),
        "valid": valid_set.shuffle() #.select(range(500))
    }
)

raw_dataset

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 20000
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 3322
    })
})

In [ ]:
for key in raw_dataset["train"][0]:
  # check the data type of the value
  value = raw_dataset['train'][0][key]
  if isinstance(value, int):
      # convert to a list or numpy array
      value = str(value)
      # or: value = np.array([value])
  print(f"{key.upper()}: {value}")

REPO_NAME: fengzhyuan/scikit-learn
PATH: sklearn/ensemble/tests/test_partial_dependence.py
COPIES: 365
SIZE: 6996
CONTENT: """
Testing for the partial dependence module.
"""

import numpy as np
from numpy.testing import assert_array_equal

from sklearn.utils.testing import assert_raises
from sklearn.utils.testing import if_matplotlib
from sklearn.ensemble.partial_dependence import partial_dependence
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import datasets


# toy sample
X = [[-2, -1], [-1, -1], [-1, -2], [1, 1], [1, 2], [2, 1]]
y = [-1, -1, -1, 1, 1, 1]
T = [[-1, -1], [2, 2], [3, 2]]
true_result = [-1, 1, 1]

# also load the boston dataset
boston = datasets.load_boston()

# also load the iris dataset
iris = datasets.load_iris()


def test_partial_dependence_classifier():
    # Test partial dependence for classifier
    clf = Gradient

# Encode Text

In [ ]:
# CHECKPOINT = 'gpt2'
# tokenizer = GPT2Tokenizer.from_pretrained(CHECKPOINT)

CHECKPOINT = 'huggingface-course/code-search-net-tokenizer'
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

In [ ]:
# tokenizer.is_fast

In [ ]:
tokenizer("What is your name?", "My name is Daniel Adama.")

{'input_ids': [25027, 300, 2827, 455, 31, 9772, 455, 300, 523, 19579, 336, 6204, 15655, 14], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Instead, we’ll use the return_overflowing_tokens option to tokenize the whole input and split it into several chunks, as we did in Chapter 6. We’ll also use the return_length option to return the length of each created chunk automatically. Often the last chunk will be smaller than the context size, and we’ll get rid of these pieces to avoid padding issues; we don’t really need them as we have plenty of data anyway.

In [ ]:
context_length = 128

outputs = tokenizer(
    raw_dataset["train"][:2]["content"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 89
Input chunk lengths: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 52, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 26]
Chunk mapping: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


We can see that we get 27 segments in total from those two examples. Looking at the chunk lengths, we can see that the chunks at the ends of both documents have less than 128 tokens (45 and 69, respectively). These represent just a small fraction of the total chunks that we have, so we can safely throw them away. With the *overflow_to_sample_mapping* field, we can also reconstruct which chunks belonged to which input samples.

In [ ]:
def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_dataset.map(
    tokenize, batched=True, remove_columns=raw_dataset["train"].column_names
)
tokenized_datasets

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3322 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 557217
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 93164
    })
})

# Initializing a new model

In [ ]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.2M parameters


In [ ]:
model.config.pad_token_id = model.config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
args = TrainingArguments(
    output_dir="codehelper-ds-v2",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-2, #5e-3
    save_steps=5_000,
    fp16=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)

In [ ]:
eval_results = trainer.evaluate()
print(f">>>Before Train Perplexity")

>>>Before Train Perplexity


In [ ]:
math.exp(eval_results['eval_loss'])

56957.891041258525

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=2176, training_loss=4014175.0588235296, metrics={'train_runtime': 6366.6513, 'train_samples_per_second': 87.521, 'train_steps_per_second': 0.342, 'total_flos': 3.6388573544448e+16, 'train_loss': 4014175.0588235296, 'epoch': 1.0})

In [ ]:
eval_results = trainer.evaluate()
print(f">>>After Train Perplexity")

>>>After Train Perplexity


In [ ]:
math.exp(eval_results['eval_loss'])

nan

In [ ]:
trainer.evaluate()

{'eval_loss': nan,
 'eval_runtime': 397.5605,
 'eval_samples_per_second': 234.339,
 'eval_steps_per_second': 7.325,
 'epoch': 1.0}

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import Repository, get_full_repo_name
api = HfApi()
username = api.whoami()['name']

MODEL_NAME = "codehelper-ds-v2"
repo_name = get_full_repo_name(MODEL_NAME)
repo_name

'danny3/codehelper-ds-v2'

In [ ]:
output_dir = MODEL_NAME
repo = Repository(output_dir, clone_from=repo_name)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn    │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                                                         │
│   121 │                                                                                          │
│   122 │   return _inner_fn  # type: ignore                                                       │
│   123                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/repository.py:516 in __init__            │
│                                                                                                  │
│    513 │   │   │   self.huggingface_token = HfFolder.get_token()                                 │
│    514 │   │                                                                                     │
│    515 │   │   if clone_from is not None:                                                        │
│ ❱  516 │   │   │   self.clone_from(repo_url=clone_from)                                          │
│    517 │   │   else:                                                                             │
│    518 │   │   │   if is_git_repo(self.local_dir):                                               │
│    519 │   │   │   │   logger.debug("[Repository] is a valid git repo")                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn    │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                                                         │
│   121 │                                                                                          │
│   122 │   return _inner_fn  # type: ignore                                                       │
│   123                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/repository.py:680 in clone_from          │
│                                                                                                  │
│    677 │   │   │   else:                                                                         │
│    678 │   │   │   │   # Check if the folder is the root of a git repository                     │
│    679 │   │   │   │   if not is_git_repo(self.local_dir):                                       │
│ ❱  680 │   │   │   │   │   raise EnvironmentError(                                               │
│    681 │   │   │   │   │   │   "Tried to clone a repository

In [ ]:
# repo.push_to_hub(
#                     commit_message="Upload version 1 Model", blocking=False
#                 )

In [ ]:
trainer.push_to_hub(
    f"{username}/{MODEL_NAME}",
    commit_message="Upload version 2 Model"
)

In [ ]:
!zip -r codehelper-ds.zip codehelper-ds

  adding: codehelper-ds/ (stored 0%)
  adding: codehelper-ds/tokenizer.json (deflated 71%)
  adding: codehelper-ds/generation_config.json (deflated 23%)
  adding: codehelper-ds/.gitignore (stored 0%)
  adding: codehelper-ds/config.json (deflated 51%)
  adding: codehelper-ds/training_args.bin (deflated 48%)
  adding: codehelper-ds/vocab.json (deflated 57%)
  adding: codehelper-ds/special_tokens_map.json (deflated 60%)
  adding: codehelper-ds/.git/ (stored 0%)
  adding: codehelper-ds/.git/HEAD (stored 0%)
  adding: codehelper-ds/.git/hooks/ (stored 0%)
  adding: codehelper-ds/.git/hooks/pre-rebase.sample (deflated 59%)
  adding: codehelper-ds/.git/hooks/pre-push (deflated 24%)
  adding: codehelper-ds/.git/hooks/pre-merge-commit.sample (deflated 39%)
  adding: codehelper-ds/.git/hooks/applypatch-msg.sample (deflated 42%)
  adding: codehelper-ds/.git/hooks/update.sample (deflated 68%)
  adding: codehelper-ds/.git/hooks/pre-receive.sample (deflated 40%)
  adding: codehelper-ds/.git/hooks/po

In [ ]:
# def s3_store_model(filePath: any, bytes: any, file: str) -> None:
#     s3.put_object(
#         Bucket=AWS_MODEL_BUCKET_NAME,
#         Key=filePath,
#         Body=bytes,
#         ContentType= file.content_type
#     )
# path = f"models/{MODEL_NAME}" 

# s3_store_model(filePath=,bytes=,file=)

In [ ]:
# Generate text from the trained model
# prompt = "Hello, how are you today?"
prompt = """\
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
"""



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

# model_path = "codehelper-ds"
# # tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = GPT2LMHeadModel.from_pretrained(model_path)

# generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
# output = generator(prompt, num_return_sequences=1)
# print(output[0]["generated_text"])

In [ ]:
# Test locally trained model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
model.config.pad_token_id = model.config.eos_token_id

prompt = '''\
"""create a dataframe"""
'''

input_ids = tokenizer.encode(prompt, return_tensors="pt")
generated_text_ids = model.generate(input_ids=input_ids, max_length=80)

generated_text = tokenizer.decode(generated_text_ids[0], skip_special_tokens=True)
print(generated_text)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:657 in     │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   654 │   │   # If that did not work, let's try to use the config.                               │
│   655 │   │   if config_tokenizer_class is None:                                                 │
│   656 │   │   │   if not isinstance(config, PretrainedConfig):                                   │
│ ❱ 657 │   │   │   │   config = AutoConfig.from_pretrained(                                       │
│   658 │   │   │   │   │   pretrained_model_name_or_path, trust_remote_code=trust_remote_code,    │
│   659 │   │   │   │   )                                                                          │
│   660 │   │   │   config_tokenizer_class = config.tokenizer_class                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:916 in    │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   913 │   │   kwargs["_from_auto"] = True                                                        │
│   914 │   │   kwargs["name_or_path"] = pretrained_model_name_or_path                             │
│   915 │   │   trust_remote_code = kwargs.pop("trust_remote_code", False)                         │
│ ❱ 916 │   │   config_dict, unused_kwargs = PretrainedConfig.get_config_dict(pretrained_model_n   │
│   917 │   │   if "auto_map" in config_dict and "AutoConfig" in config_dict["auto_map"]:          │
│   918 │   │   │   if not trust_remote_code:                                                      │
│   919 │   │   │   │   raise ValueError(                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:573 in               │
│ get_config_dict                                                                                  │
│                                                                                                  │
│   570 │   │   """                                                                                │
│   571 │   │   original_kwargs = copy.deepcopy(kwargs)                                            │
│   572 │   │   # Get config dict associated with the base config file                             │
│ ❱ 573 │   │   config_dict, kwargs = cls._get_config_dict(pretrained_model_name_or_path, **kwar   │
│   574 │   │   if "_commit_hash" in config_dict:                                                  │
│   575 │   │   │   original_kwargs["_commit_hash"] = config_dict["_commit_hash"]                  │
│   576                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:628 in               │
│ _get_config_dict                                                                                 │
│                                                                                                  │
│   625 │   │   │                                            

In [ ]:
import torch
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model=f"{username}/codehelper-ds", device=device
)

In [ ]:
prompt = '''\
"""create some data"""
x = np.random.randn(100)
y = np.random.randn(100)

"""function to add x and y"""
'''

print(pipe(prompt, num_return_sequences=1, max_length=100, do_sample=True)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# import random forest regressor from scikit-learn
from sklearn.ensemble import RandomForestRegressor

# fit random forest model with 300 estimators on X, y:
# X, y = digits.data, digits.target
# x, y = np.arange(50) - 1, 0.3, 0.3, 0.7]

y_test = [(X[y!= 1] | 'none')]

# plot result
c = clf.


In [ ]:
# from transformers import AutoTokenizer, AutoModelWithLMHead

# tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot")
# model = AutoModelWithLMHead.from_pretrained("codeparrot/codeparrot")

# inputs = tokenizer("def hello_world():", return_tensors="pt")
# outputs = model(**inputs)

In [ ]:
# model.config.pad_token_id = model.config.eos_token_id

In [ ]:
keytoken_ids = []
for keyword in [
    "plt",
    "pd",
    "sk",
    "fit",
    "predict",
    " plt",
    " pd",
    " sk",
    " fit",
    " predict",
    "testtest",
]:
    ids = tokenizer([keyword]).input_ids[0]
    if len(ids) == 1:
        keytoken_ids.append(ids[0])
    else:
        print(f"Keyword has not single token: {keyword}")

Keyword has not single token: testtest


In [ ]:
from torch.nn import CrossEntropyLoss
import torch
from torch.utils.data.dataloader import DataLoader


def keytoken_weighted_loss(inputs, logits, keytoken_ids, alpha=1.0):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduce=False)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    # Calculate and scale weighting
    weights = torch.stack([(inputs == kt).float() for kt in keytoken_ids]).sum(
        axis=[0, 2]
    )
    weights = alpha * (1.0 + weights)
    # Calculate weighted average
    weighted_loss = (loss_per_sample * weights).mean()
    return weighted_loss


tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=32, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets["valid"], batch_size=32)


weight_decay = 0.1


def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch["input_ids"], labels=batch["input_ids"])

        losses.append(accelerator.gather(outputs.loss))
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [ ]:
from accelerate import Accelerator
from torch.optim import AdamW
from transformers import get_scheduler

model = GPT2LMHeadModel(config)
optimizer = AdamW(get_grouped_params(model), lr=5e-4)
accelerator = Accelerator()#fp16=True)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=1_000,
    num_training_steps=num_training_steps,
)

In [ ]:
evaluate()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in evaluate:53                                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated

In [ ]:
from tqdm.notebook import tqdm

gradient_accumulation_steps = 8
eval_steps = 5_000

model.train()
completed_steps = 0
for epoch in range(num_train_epochs):
    for step, batch in tqdm(
        enumerate(train_dataloader, start=1), total=num_training_steps
    ):
        logits = model(batch["input_ids"]).logits
        loss = keytoken_weighted_loss(batch["input_ids"], logits, keytoken_ids)
        if step % 100 == 0:
            accelerator.print(
                {
                    "lr": get_lr(),
                    "samples": step * samples_per_step,
                    "steps": completed_steps,
                    "loss/train": loss.item() * gradient_accumulation_steps,
                }
            )
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        if step % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
        if (step % (eval_steps * gradient_accumulation_steps)) == 0:
            eval_loss, perplexity = evaluate()
            accelerator.print({"loss/eval": eval_loss, "perplexity": perplexity})
            model.train()
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
            if accelerator.is_main_process:
                tokenizer.save_pretrained(output_dir)
                # repo.push_to_hub(
                #     commit_message=f"Training in progress step {step}", blocking=False
                # )